<a href="https://colab.research.google.com/github/ThuanPhong0126/CS114.K21/blob/master/FolioRandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pwd

'/content'

In [0]:
cd /content/drive/My Drive/Colab Notebooks/Folio/Folio Leaf Dataset/Folio Leaf Dataset

/content/drive/My Drive/Colab Notebooks/Folio/Folio Leaf Dataset/Folio Leaf Dataset


In [0]:
pwd

'/content/drive/My Drive/Colab Notebooks/Folio/Folio Leaf Dataset/Folio Leaf Dataset'

In [0]:
!pip install mahotas

     |████████████████████████████████| 1.5MB 2.8MB/s 
  Created wheel for mahotas: filename=mahotas-1.4.9-cp36-cp36m-linux_x86_64.whl size=4265618 sha256=e144799f35778020fad0bfa68304d074c8c990d5e51c4f0f5654dde7282a7b8c
  Stored in directory: /root/.cache/pip/wheels/02/e7/50/b67eee3b19c479cc5b41f10230bf9c1556be6eaaf095c6e997
Successfully built mahotas


In [0]:
import cv2
import os
import glob
import numpy as np
import mahotas

In [0]:
bins=8

In [0]:
def fd_histogram(image,mask=None):
  image=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
  hist=cv2.calcHist([image],[0,1,2],None,[bins,bins,bins],[0,256,0,256,0,256])
  cv2.normalize(hist,hist)
  return hist.flatten()

In [0]:
def fd_hu_moments(image):
  image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  feature=cv2.HuMoments(cv2.moments(image)).flatten()
  return feature

In [0]:
def fd_haralick(image):
    # chuyển về ảnh gray
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick

In [0]:
f_labels=os.listdir('Folio')

In [0]:
sl=0
labels=[]
global_features=[]
for dir_path in glob.glob('Folio/*'):
  current_label=dir_path.split('/')[-1]
  for img in glob.glob(os.path.join(dir_path,'*')):
    image=cv2.imread(img)
    image=cv2.resize(image,(100,100))
    sl=sl+1
    fv_hu_moments = fd_hu_moments(image)
    fv_haralick   = fd_haralick(image)
    fv_histogram  = fd_histogram(image)
    global_feature = np.hstack([fv_histogram, fv_hu_moments, fv_haralick])
    labels.append(current_label)
    global_features.append(global_feature)

In [0]:
sl

637

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(global_features,labels,test_size=0.3,random_state=101)

In [0]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import h5py

In [0]:
# get the overall feature vector size
print ("[STATUS] feature vector size {}".format(np.array(X_train).shape))

# get the overall training label size
print ("[STATUS] training Labels {}".format(np.array(y_train).shape))

# encode the target labels
le = LabelEncoder()
target = le.fit_transform(y_train)

# normalize the feature vector in the range (0-1)
scaler = MinMaxScaler(feature_range=(0, 1))
rescaled_features = scaler.fit_transform(X_train)

# save the feature vector using HDF5
h5f_data = h5py.File('data.h5', 'w')
h5f_data.create_dataset('dataset_1', data=np.array(rescaled_features))

h5f_label = h5py.File('labels.h5', 'w')
h5f_label.create_dataset('dataset_1', data=np.array(target))

h5f_data.close()
h5f_label.close()

print("[STATUS] end of training..")

[STATUS] feature vector size (445, 532)
[STATUS] training Labels (445,)
[STATUS] end of training..


In [0]:
num_trees=300

In [0]:
h5f_data = h5py.File('data.h5', 'r')
h5f_label = h5py.File('labels.h5', 'r')

global_features_string = h5f_data['dataset_1']
global_labels_string = h5f_label['dataset_1']

global_features = np.array(global_features_string)
global_labels = np.array(global_labels_string)

h5f_data.close()
h5f_label.close()

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
clf  = RandomForestClassifier(n_estimators=num_trees)
clf.fit(global_features, global_labels)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
le=LabelEncoder()

In [0]:
y_result=le.fit_transform(y_test)
y_pred=clf.predict(X_test)

In [0]:
print("Result: ", (y_pred == y_result).tolist().count(True)/len(y_result))

Result:  0.4791666666666667


In [0]:
from sklearn.metrics import confusion_matrix,classification_report

In [0]:
print(confusion_matrix(y_result,y_pred))

[[4 0 0 ... 0 0 0]
 [0 7 0 ... 0 0 0]
 [0 4 0 ... 0 0 0]
 ...
 [0 0 0 ... 7 0 0]
 [0 0 0 ... 0 5 0]
 [1 0 0 ... 0 0 2]]


In [0]:
print(classification_report(y_result,y_pred))

              precision    recall  f1-score   support

           0       0.40      1.00      0.57         4
           1       0.25      1.00      0.40         7
           2       0.00      0.00      0.00         8
           3       0.12      0.75      0.20         4
           4       0.17      1.00      0.29         6
           5       1.00      0.67      0.80         3
           6       0.00      0.00      0.00         5
           7       0.60      1.00      0.75         3
           8       0.50      0.40      0.44         5
           9       0.27      0.75      0.40         4
          10       0.57      0.80      0.67         5
          11       1.00      1.00      1.00         7
          12       0.86      0.75      0.80         8
          13       0.00      0.00      0.00         7
          14       1.00      0.50      0.67         6
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00         7
          17       1.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
